Load the libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import psycopg2
import yaml
import sys
import json
import csv
import re

from Bio import SeqIO

Read the contents of the yaml file

In [ ]:
stream = open("db.yml", "r")
db = yaml.load(stream)
stream.close()

In [ ]:
# Print the experiment title

# Print the experiment title

expName  = "Mosquito"
dataPath = "Data/"+ expName
print (dataPath)

Insert into the *experiment* table:

In [ ]:
try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    expINS = "INSERT INTO experiment (title) VALUES ('Mosquito') " 
    cur.execute(expINS,)
    con.commit()
        
except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

Insert into the *sample* table:

In [ ]:
try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()
    
    # We cannot have a duplicate name and exp_id ( AND exp_id = %)
    smpINS = 'INSERT INTO sample (name, exp_id) SELECT %s, %s WHERE NOT EXISTS (SELECT id FROM sample WHERE name = %s AND exp_id = %s);' 
    
    path = dataPath + "/AminoAcids-or-ORFs-orTGEs"
    
    for i in os.listdir(path):
        if i.endswith(".assemblies.fasta.transdecoder.pep.identified.fasta"): 
            print(i)
            sampleName = str(i[0:i.find('.')])
            
            query = ("SELECT id FROM experiment WHERE title = 'Mosquito'")
            cur.execute(query)
            expID = cur.fetchone()[0]
            
            cur.execute(smpINS, (sampleName, expID, sampleName, expID))
            con.commit()
        
except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

Load the data into the *tge* table

In [ ]:
## Load the data into the <tge> table

try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()
    
    # We cannot have duplicate TGE amino acid sequences 
    tgeINS = 'INSERT INTO tge (amino_seq) SELECT %s WHERE NOT EXISTS (SELECT id FROM tge WHERE amino_seq = %s);' 
    
    tgePath = dataPath + "/AminoAcids-or-ORFs-orTGEs/"
        
    for i in os.listdir(tgePath):
        if i.endswith(".assemblies.fasta.transdecoder.pep.identified.fasta"): 
            print(i)
             
            for input in SeqIO.parse(tgePath+i, "fasta"):
                tgeSeq = input.seq
                tgeSeq = str(tgeSeq[0:tgeSeq.find('*')])
                cur.execute(tgeINS, (tgeSeq, tgeSeq))
                con.commit()

except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
## Load the data into the <observation> table

try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()
    
    tgeINS = 'INSERT INTO observation (tge_id, sample_id, name, description) SELECT %s , %s , %s, %s WHERE NOT EXISTS (SELECT id FROM observation WHERE description = %s AND sample_id = %s);' 
    
    tgePath = dataPath + "/AminoAcids-or-ORFs-orTGEs/"
        
    for i in os.listdir(tgePath):
        if i.endswith(".assemblies.fasta.transdecoder.pep.identified.fasta"): 
            print (i)
            
            sampleName = str(i[0:i.find('.')])
            # WE SHOULD TAKE THE EXPERIMENT NAME TOO 
            query = ("SELECT id FROM sample WHERE sample.name = '"+sampleName+"'")
            cur.execute(query)
            sampleID = cur.fetchone()[0]
            
            for input in SeqIO.parse(tgePath+i, "fasta"):
                tgeName  = input.id
                tgeName  = tgeName[0:tgeName.find('|')]
                
                tgeSeq = input.seq
                tgeSeq = str(tgeSeq[0:tgeSeq.find('*')])
            
                query = ("SELECT id FROM tge WHERE tge.amino_seq = '"+tgeSeq+"'")
                cur.execute(query)
                tgeID = cur.fetchone()[0]
            
                cur.execute(tgeINS, (tgeID, sampleID, tgeName, input.id, input.id, sampleID))
                con.commit()
                
except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
## Load the data into the <transcript> table

try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    trsINS = 'INSERT INTO transcript (obs_id, dna_seq) SELECT %s ,%s WHERE NOT EXISTS (SELECT id FROM transcript WHERE obs_id = %s AND dna_seq  = %s) AND EXISTS (SELECT id FROM observation WHERE name = %s AND sample_id = %s);'
    
    trsPath = dataPath + "/transcripts/"
    
    for i in os.listdir(trsPath):
        if i.endswith(".assemblies.fasta.identified.fasta"): 
            print (i)
            
            sampleName = str(i[0:i.find('.')])
            query = ("SELECT id FROM sample WHERE sample.name = '"+sampleName+"'")
            cur.execute(query)
            sampleID = cur.fetchone()[0]
            
            for input in SeqIO.parse(trsPath+i, "fasta"):
                trsName = input.id
                trsSeq  = str(input.seq)
                
                query = ("SELECT id FROM observation WHERE name = '"+trsName+"' AND sample_id="+str(sampleID))
                cur.execute(query)
                tgeObsID = cur.fetchone()[0]

                cur.execute(trsINS, (tgeObsID, trsSeq, tgeObsID, trsSeq, trsName, sampleID))
                con.commit()
        

except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
## update annotation in observation

try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    tgeALT = 'UPDATE observation SET organism = %s, uniprot_id = %s, protein_name= %s, protein_descr = %s, gene_name = %s, tge_class = %s, variation = %s, long_description = %s  WHERE description = %s AND sample_id = %s'
    
    annPath = dataPath + "/Annotation/"
    
    for i in os.listdir(annPath):
        if i.endswith(".pep_details_annotation.csv"): 
            print (i)
            
            sampleName = str(i[0:i.find('.')])
            query = ("SELECT id FROM sample WHERE sample.name = '"+sampleName+"'")
            
            cur.execute(query)
            sampleID = cur.fetchone()[0]
            
            annotation = pd.read_csv(annPath+i)
                
            for i in range(len(annotation)): 
                descr  = annotation['ORF Id'][i]
                tgeDescr  = descr[0:descr.find(' ')]
                
                uniprotID = annotation['Protein ID'][i]
#                 m = re.search("(?<=\|).*?(?=\|)", protID)
                if uniprotID is None:
                    uniprotID = '-'
            
                proteinName  = str(annotation['Protein Name'][i])
                geneName     = str(annotation['Gene Name'][i])
                proteinDescr = str(annotation['Protein description'][i])
                tgeClass  = str(annotation['Class'][i])
                variation = int(annotation['Variation'][i])
                species   = str(annotation['Species'][i])
                
#                 print(descr)
#                 print(tgeDescr)
#                 print(uniprotID)
#                 print(proteinName)
#                 print(geneName)
#                 print(proteinDescr)
#                 print(tgeClass)
#                 print(variation)
#                 print(species)
                
                cur.execute(tgeALT, (species, uniprotID, proteinName, proteinDescr, geneName, tgeClass, variation, descr, tgeDescr, sampleID))
                con.commit()

                
except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
## update observation table with peptide number 

try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    tgeALT = 'UPDATE observation SET peptide_num = %s WHERE description = %s AND sample_id = %s'
    
    pepPath = dataPath + "/PSMs-Peptides-ORFs/"
    

    for i in os.listdir(pepPath):
        if i.endswith("+fdr+th+grouping+prt_filtered.csv"): 
            print (i)
            
            sampleName = str(i[0:i.find('+')])
            query = ("SELECT id FROM sample WHERE sample.name = '"+sampleName+"'")
            cur.execute(query)
            sampleID = cur.fetchone()[0]
            
            tge = pd.read_csv(pepPath+i)

            for i in range(len(tge)): 
                descr  = str(tge['description'][i])
                tgeID  = descr[0:descr.find(' ')]
                pepNum = int(tge['distinct peptide sequences'][i])
                
                cur.execute(tgeALT, (pepNum, tgeID, sampleID))
                con.commit()

except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
## Insert into peptide 


try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    insPep = 'INSERT INTO peptide (aa_seq) SELECT %s WHERE NOT EXISTS (SELECT id FROM peptide WHERE aa_seq = %s);'
    
    pepPath = dataPath + "/PSMs-Peptides-ORFs/"
    

    for i in os.listdir(pepPath):
        if i.endswith("+fdr+th+grouping_filtered.csv"): 
            print (i)
            
            peptides = pd.read_csv(pepPath+i)

            for i in range(len(peptides)): 
                m = re.search("(?<=index=).*?$", peptides['Spectrum ID'][i])
                specID = m.group(0)
                
                amino   = peptides['Sequence'][i]
#                 calc_mz = peptides['Calc m/z'][i]
#                 exp_mz  = peptides['Exp m/z'][i]
                
                cur.execute(insPep, (amino, amino))
                con.commit()
    
except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
## Peptides for each TGE


try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    insTgePep = 'INSERT INTO tge_peptide (obs_id, peptide_id) SELECT %s , %s WHERE NOT EXISTS (SELECT id FROM tge_peptide WHERE obs_id = %s AND peptide_id  = %s);'
    
    pepPath = dataPath + "/PSMs-Peptides-ORFs/"
    

    for i in os.listdir(pepPath):
        if i.endswith("+fdr+th+grouping_filtered.csv"): 
            print (i)
            
            sampleName = str(i[0:i.find('+')])
            query = ("SELECT id FROM sample WHERE sample.name = '"+sampleName+"'")
            cur.execute(query)
            sampleID = cur.fetchone()[0]
            
            peptides = pd.read_csv(pepPath+i)
             
            
            for i in range(len(peptides)): 
#                 m = re.search("(?<=index=).*?$", peptides[i][0])
#                 specID = m.group(0)

                amino   = peptides['Sequence'][i]
                
                query = ("SELECT id FROM peptide WHERE aa_seq = '"+amino+"'") 
                cur.execute(query)
                pepID = cur.fetchone()[0]
                
                proteinacc = peptides['proteinacc_start_stop_pre_post_;'][i]
                arr =  proteinacc.split(';')
                
                for x in range(0, len(arr)):
                    n=2
                    tge = arr[x]                    
                    m=re.match(r'^((?:[^_]*_){%d}[^_]*)_(.*)' % (n-1), tge)
                    tgeDescr = m.groups()[0]
                    
                    query = ("SELECT id FROM observation WHERE description = '"+tgeDescr+"' AND sample_id = "+str(sampleID))
                    cur.execute(query)
                    tgeID = cur.fetchone()
                    
                    if tgeID:
                        tgeID = tgeID[0]
                        
                        cur.execute(insTgePep, (tgeID, pepID,tgeID, pepID ))
                        con.commit()

except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
## Insert into psm 

try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    insSpec = 'INSERT INTO psm (peptide_id, name, spectrum_id, title, location, retention, calc_mz, exp_mz, charge, modifications, raw_score, denovo_score, spec_evalue, evalue, qvalue, pep_qvalue, local_fdr, q_value, fdr_score ) SELECT %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s WHERE NOT EXISTS (SELECT id FROM psm WHERE spectrum_id = %s);'
    
    pepPath = dataPath + "/PSMs-Peptides-ORFs/"
    

    for i in os.listdir(pepPath):
        if i.endswith(".assemblies.fasta.transdecoder.pep+fdr+th+grouping_filtered.csv"): 
            print i
            
            with open(pepPath+i,'rb') as fin: 
                dr = csv.DictReader(fin)
                psms = [( i['PSM_ID'], i['Spectrum ID'], i['Spectrum Title'], i['Raw data location'], 
                            i['Retention Time (s)'], i['Calc m/z'], i['Exp m/z'], i['Charge'], i['Modifications'],
                            i['MS-GF:RawScore'], i['MS-GF:DeNovoScore'], i['MS-GF:SpecEValue'], i['MS-GF:EValue'], 
                            i['MS-GF:QValue'], i['MS-GF:PepQValue'], i['PSM-level local FDR'], i['PSM-level q-value'], 
                            i['PSM-level FDRScore'], i['Sequence']) for i in dr]

            for i in range(len(psms)): 
                location  = psms[i][3]
                
                m = re.search("(?<=index=).*?$", psms[i][1])
                specID = m.group(0)
                
                specTitle = psms[i][2]
                
                amino   = psms[i][18]
                
                query = ("SELECT id FROM peptide WHERE aa_seq = '"+amino+"'") 
                cur.execute(query)
                pepID = cur.fetchone()[0]
                
                cur.execute(insSpec, (pepID, psms[i][0], specID, specTitle, location, psms[i][4], psms[i][5], psms[i][6], psms[i][7], psms[i][8], psms[i][9], psms[i][10], psms[i][11], psms[i][12], psms[i][13], psms[i][14], psms[i][15], psms[i][16], psms[i][17], specID))
                con.commit()
    
except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")

In [ ]:
import vcf


try:
    con = psycopg2.connect(database=db["test"]["database"], user=db["test"]["username"], host=db["test"]["host"], password=db["test"]["password"])
    cur = con.cursor()

    insSpec = 'INSERT INTO variation (obs_id, chrom, pos, alt, qual, var_type, qpos, peptide_num, peptides) SELECT %s, %s, %s, %s, %s, %s, %s, %s, %s;'
    
    variationPath = dataPath + "/Variations-proVCF/"
    

    for filename in os.listdir(pepPath):
        if filename.endswith(".assemblies.fasta.transdecoder.pep_pepEvd.vcf"): 
            print filename
            
            vcf_reader = vcf.Reader(open(variationPath+filename, 'r'))

            for record in vcf_reader:
                record = next(vcf_reader)
                print record.CHROM
                print record.ALT
                print record.QUAL
                print record.INFO['SubjectId']
                print record.INFO['Alignment'][0]
                print record.INFO['Type']
                print record.INFO['QPOS']
                print record.INFO['PeptideCount']
                print record.INFO['UniquePeptideCount']
                print record.INFO['Score']
            
except (psycopg2.DatabaseError, e):
    if con:
        con.rollback()
    
    print('Error %s' % e)    
    sys.exit(1)
    
finally:
    if con:
        con.close()
        print("Connection closed")"
